In [2]:
import numpy as np
import pandas as pd

In [3]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import gc
import lightgbm as lgb
import time
# import datetime
# import xgboost as xgb
# import time
# import itertools
# from sklearn.linear_model import LinearRegression
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import OneHotEncoder
# from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score


In [5]:
INPUT_DIR = '/kaggle/input/m5-forecasting-accuracy'

calendar_df = pd.read_csv(f"{INPUT_DIR}/calendar.csv")
sell_prices_df = pd.read_csv(f"{INPUT_DIR}/sell_prices.csv")
sales_train_validation_df = pd.read_csv(f"{INPUT_DIR}/sales_train_validation.csv")
sample_submission_df = pd.read_csv(f"{INPUT_DIR}/sample_submission.csv")

In [ ]:
calendar_df.head()

In [6]:
# Calendar data type cast -> Memory Usage Reduction
calendar_df[["month", "snap_CA", "snap_TX", "snap_WI", "wday"]] = calendar_df[["month", "snap_CA", "snap_TX", "snap_WI", "wday"]].astype("int8")
calendar_df[["wm_yr_wk", "year"]] = calendar_df[["wm_yr_wk", "year"]].astype("int16") 
calendar_df["date"] = calendar_df["date"].astype("datetime64")

nan_features = ['event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']
for feature in nan_features:
    calendar_df[feature].fillna('unknown', inplace = True)

calendar_df[["weekday", "event_name_1", "event_type_1", "event_name_2", "event_type_2"]] = calendar_df[["weekday", "event_name_1", "event_type_1", "event_name_2", "event_type_2"]] .astype("category")

In [ ]:
calendar_df.head()

In [8]:
sales_train_validation_df.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4


In [7]:
# Sales Training dataset cast -> Memory Usage Reduction
sales_train_validation_df.loc[:, "d_1":] = sales_train_validation_df.loc[:, "d_1":].astype("int16")

In [9]:
sell_prices_df.head()

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26


In [10]:
# Make ID column to sell_price dataframe
sell_prices_df.loc[:, "id"] = sell_prices_df.loc[:, "item_id"] + "_" + sell_prices_df.loc[:, "store_id"] + "_validation"

In [11]:
sell_prices_df.head()

,store_id,item_id,wm_yr_wk,sell_price,id
0,CA_1,HOBBIES_1_001,11325,9.58,HOBBIES_1_001_CA_1_validation
1,CA_1,HOBBIES_1_001,11326,9.58,HOBBIES_1_001_CA_1_validation
2,CA_1,HOBBIES_1_001,11327,8.26,HOBBIES_1_001_CA_1_validation
3,CA_1,HOBBIES_1_001,11328,8.26,HOBBIES_1_001_CA_1_validation
4,CA_1,HOBBIES_1_001,11329,8.26,HOBBIES_1_001_CA_1_validation


In [12]:
sell_prices_df = pd.concat([sell_prices_df, sell_prices_df["item_id"].str.split("_", expand=True)], axis=1)
sell_prices_df.head()

,store_id,item_id,wm_yr_wk,sell_price,id,0,1,2
0,CA_1,HOBBIES_1_001,11325,9.58,HOBBIES_1_001_CA_1_validation,HOBBIES,1,001
1,CA_1,HOBBIES_1_001,11326,9.58,HOBBIES_1_001_CA_1_validation,HOBBIES,1,001
2,CA_1,HOBBIES_1_001,11327,8.26,HOBBIES_1_001_CA_1_validation,HOBBIES,1,001
3,CA_1,HOBBIES_1_001,11328,8.26,HOBBIES_1_001_CA_1_validation,HOBBIES,1,001
4,CA_1,HOBBIES_1_001,11329,8.26,HOBBIES_1_001_CA_1_validation,HOBBIES,1,001


In [13]:
sell_prices_df = sell_prices_df.rename(columns={0:"cat_id", 1:"dept_id"})
sell_prices_df.head()

,store_id,item_id,wm_yr_wk,sell_price,id,cat_id,dept_id,2
0,CA_1,HOBBIES_1_001,11325,9.58,HOBBIES_1_001_CA_1_validation,HOBBIES,1,001
1,CA_1,HOBBIES_1_001,11326,9.58,HOBBIES_1_001_CA_1_validation,HOBBIES,1,001
2,CA_1,HOBBIES_1_001,11327,8.26,HOBBIES_1_001_CA_1_validation,HOBBIES,1,001
3,CA_1,HOBBIES_1_001,11328,8.26,HOBBIES_1_001_CA_1_validation,HOBBIES,1,001
4,CA_1,HOBBIES_1_001,11329,8.26,HOBBIES_1_001_CA_1_validation,HOBBIES,1,001


In [14]:
sell_prices_df[["store_id", "item_id", "cat_id", "dept_id"]] = sell_prices_df[["store_id","item_id", "cat_id", "dept_id"]].astype("category")
sell_prices_df = sell_prices_df.drop(columns=2)
sell_prices_df.head()

,store_id,item_id,wm_yr_wk,sell_price,id,cat_id,dept_id
0,CA_1,HOBBIES_1_001,11325,9.58,HOBBIES_1_001_CA_1_validation,HOBBIES,1
1,CA_1,HOBBIES_1_001,11326,9.58,HOBBIES_1_001_CA_1_validation,HOBBIES,1
2,CA_1,HOBBIES_1_001,11327,8.26,HOBBIES_1_001_CA_1_validation,HOBBIES,1
3,CA_1,HOBBIES_1_001,11328,8.26,HOBBIES_1_001_CA_1_validation,HOBBIES,1
4,CA_1,HOBBIES_1_001,11329,8.26,HOBBIES_1_001_CA_1_validation,HOBBIES,1


DATA CLEANING

```
First, let's combine all three dataframe.
The important thing is changing data format from wide to long to make prediction model easier
(Though this notebook doesn't dive into predicition model itself.)
```



In [ ]:
#wide to long meaning
np.random.seed(123)
df = pd.DataFrame({"A1970" : {0 : "a", 1 : "b", 2 : "c"},
                   "A1980" : {0 : "d", 1 : "e", 2 : "f"},
                   "B1970" : {0 : 2.5, 1 : 1.2, 2 : .7},
                   "B1980" : {0 : 3.2, 1 : 1.3, 2 : .1},
                   "X"     : dict(zip(range(3), np.random.randn(3)))
                  })
df["id"] = df.index
print(df)
 # A1970 A1980  B1970  B1980         X  id
#0     a     d    2.5    3.2 -1.085631   0
#1     b     e    1.2    1.3  0.997345   1
#2     c     f    0.7    0.1  0.282978   2
pd.wide_to_long(df, ["A", "B"], i="id", j="year")
#
#               X  A    B
#id year
#0  1970 -1.085631  a  2.5
#1  1970  0.997345  b  1.2
#2  1970  0.282978  c  0.7
#0  1980 -1.085631  d  3.2
#1  1980  0.997345  e  1.3
#2  1980  0.282978  f  0.1

In [15]:
sales_train_validation_df.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4


In [16]:
copy1=sales_train_validation_df.drop(columns=["item_id", "dept_id", "cat_id", "state_id","store_id", "id"])
copy1.head()

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,0,0,0,0,0,0,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,0,0,0,0,0,0,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,0,0,0,0,0,0,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4


In [17]:
# Wide format dataset 
df_wide_train = sales_train_validation_df.drop(columns=["item_id", "dept_id", "cat_id", "state_id","store_id", "id"]).T
print(df_wide_train.head())
df_wide_train.index = calendar_df["date"][:1913]
print(df_wide_train.head())
df_wide_train.columns = sales_train_validation_df["id"]
print(df_wide_train.head())

     0      1      2      3      4      5      6      7      8      9      \
d_1      0      0      0      0      0      0      0     12      2      0   
d_2      0      0      0      0      0      0      0     15      0      0   
d_3      0      0      0      0      0      0      0      0      7      1   
d_4      0      0      0      0      0      0      0      0      3      0   
d_5      0      0      0      0      0      0      0      0      0      0   

     ...  30480  30481  30482  30483  30484  30485  30486  30487  30488  30489  
d_1  ...      0     14      1      0      4      0      0      0      0      0  
d_2  ...      0     11      1      0      4      0      0      6      0      0  
d_3  ...      0      5      1      0      2      2      0      0      0      0  
d_4  ...      0      6      1      0      5      2      0      2      0      0  
d_5  ...      0      5      1      0      2      0      0      2      0      0  

[5 rows x 30490 columns]
            0      1     

In [18]:
# Making test label dataset
df_wide_test = pd.DataFrame(np.zeros(shape=(56, len(df_wide_train.columns))), index=calendar_df.date[1913:], columns=df_wide_train.columns) #28days of evaluation and another 28 days of validation
#print(df_wide_test.head())
df_wide = pd.concat([df_wide_train, df_wide_test])
print(df_wide_test.head())


id          HOBBIES_1_001_CA_1_validation  HOBBIES_1_002_CA_1_validation  \
date                                                                       
2016-04-25                            0.0                            0.0   
2016-04-26                            0.0                            0.0   
2016-04-27                            0.0                            0.0   
2016-04-28                            0.0                            0.0   
2016-04-29                            0.0                            0.0   

id          HOBBIES_1_003_CA_1_validation  HOBBIES_1_004_CA_1_validation  \
date                                                                       
2016-04-25                            0.0                            0.0   
2016-04-26                            0.0                            0.0   
2016-04-27                            0.0                            0.0   
2016-04-28                            0.0                            0.0   
2016-04-29 

In [19]:
# Convert wide format to long format
df_long = df_wide.stack().reset_index(1)
df_long.columns = ["id", "value"]

print(df_long.head())
del df_wide_train, df_wide_test, df_wide
gc.collect()

                                       id  value
date                                            
2011-01-29  HOBBIES_1_001_CA_1_validation    0.0
2011-01-29  HOBBIES_1_002_CA_1_validation    0.0
2011-01-29  HOBBIES_1_003_CA_1_validation    0.0
2011-01-29  HOBBIES_1_004_CA_1_validation    0.0
2011-01-29  HOBBIES_1_005_CA_1_validation    0.0


92

Stacking


In [ ]:
df_single_level_cols = pd.DataFrame([[0, 2], [3, 4]],
                                    index=['deer', 'monkey'],
                                    columns=['weight', 'height'])
print(df_single_level_cols)
print(df_single_level_cols.stack())

Back to main

In [22]:
df_long.head(25)

,id,value
date,,
2011-01-29,HOBBIES_1_001_CA_1_validation,0.0
2011-01-29,HOBBIES_1_002_CA_1_validation,0.0
2011-01-29,HOBBIES_1_003_CA_1_validation,0.0
2011-01-29,HOBBIES_1_004_CA_1_validation,0.0
2011-01-29,HOBBIES_1_005_CA_1_validation,0.0
2011-01-29,HOBBIES_1_006_CA_1_validation,0.0
2011-01-29,HOBBIES_1_007_CA_1_validation,0.0
2011-01-29,HOBBIES_1_008_CA_1_validation,12.0
2011-01-29,HOBBIES_1_009_CA_1_validation,2.0


In [ ]:
calendar_df.head()

In [ ]:
sell_prices_df.head()

In [23]:
df = pd.merge(pd.merge(df_long.reset_index(), calendar_df, on="date"), sell_prices_df, on=["id", "wm_yr_wk"]) #Reset the index, or a level of it. Reset the index of the DataFrame, and use the default one instead. If the DataFrame has a MultiIndex, this method can remove one or more levels.
df = df.drop(columns=["d"])
#df[["cat_id", "store_id", "item_id", "id", "dept_id"]] = df[["cat_id"", store_id", "item_id", "id", "dept_id"]].astype("category")
#df["sell_price"] = df["sell_price"].astype("float16")   
#df["value"] = df["value"].astype("int32")
#df["state_id"] = df["store_id"].str[:2].astype("category")

del df_long
gc.collect()

0

In [24]:
df.head(10)
#df = make_dataframe()

,date,id,value,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,store_id,item_id,sell_price,cat_id,dept_id
0,2011-01-29,HOBBIES_1_008_CA_1_validation,12.0,11101,Saturday,1,1,2011,unknown,unknown,unknown,unknown,0,0,0,CA_1,HOBBIES_1_008,0.46,HOBBIES,1
1,2011-01-30,HOBBIES_1_008_CA_1_validation,15.0,11101,Sunday,2,1,2011,unknown,unknown,unknown,unknown,0,0,0,CA_1,HOBBIES_1_008,0.46,HOBBIES,1
2,2011-01-31,HOBBIES_1_008_CA_1_validation,0.0,11101,Monday,3,1,2011,unknown,unknown,unknown,unknown,0,0,0,CA_1,HOBBIES_1_008,0.46,HOBBIES,1
3,2011-02-01,HOBBIES_1_008_CA_1_validation,0.0,11101,Tuesday,4,2,2011,unknown,unknown,unknown,unknown,1,1,0,CA_1,HOBBIES_1_008,0.46,HOBBIES,1
4,2011-02-02,HOBBIES_1_008_CA_1_validation,0.0,11101,Wednesday,5,2,2011,unknown,unknown,unknown,unknown,1,0,1,CA_1,HOBBIES_1_008,0.46,HOBBIES,1


In [ ]:
df.shape

In [ ]:
def add_date_feature(df):
    df["year"] = df["date"].dt.year.astype("int16")
    df["month"] = df["date"].dt.month.astype("int8")
    df["week"] = df["date"].dt.week.astype("int8")
    df["day"] = df["date"].dt.day.astype("int8")
    df["quarter"]  = df["date"].dt.quarter.astype("int8")
    return df

In [ ]:
df = add_date_feature(df)
df.head()

In [ ]:
df.shape